## Numpy Version

The graph is fully connected with edge weight 1/N. Each column sums to 1. In order to unbalance the graph in favor of your neighbor a node subtracts from his inloop edge and attributes this offset to his connection. For example:

n_i --> n_i 0.9
n_i --> n_j 0.1

Mii = 1/N - (1/N) * (1 - 0.9)
Mij = 1/N + (1/N) * 0.1




In [92]:
s = np.array([1.0, 10.0, 1.0, 1.0, 1.0])
ns = s / np.linalg.norm(s, 1)
print (ns)



[0.07142857 0.71428571 0.07142857 0.07142857 0.07142857]


In [132]:
import numpy as np

def boltrank(M, s):    
    v = s
    ns = s / np.linalg.norm(s, 1)
    for _ in range(1000):
        v = np.matmul(M, v)
        #v = v / np.linalg.norm(v, 1)        
    v = np.multiply(v, s)
    v = v / np.linalg.norm(v, 1)
    attr = v
    e = np.sum(s) * 0.01
    v = np.multiply(v, e)
    v = s + v
    return v, attr



# Edge matrix.
N = 5.0
ea = 1/N - (1/N) * (1 - 0.8)
eb = 1/N + (1/N) * 0.2

print (ea*10)


M = np.array([[1/N, 1/N, 1/N, 1/N, 1/N],
              [1/N, ea, 1/N, 1/N, 1/N], 
              [1/N, eb, 1/N, 1/N, 1/N], 
              [1/N, 1/N, 1/N, ea, 1/N],
              [1/N, 1/N, 1/N, eb, 1/N]], dtype=float)

print (M)

# Initial stake vector.
s = np.array([1.0, 10.0, 1.0, 1.0, 1.0])

# Run 100 steps.
steps = 100
s_init = s
print ('Initial stake:', s_init)
for i in range(steps):
    s, attr = boltrank(M, s)
    print ('Stake at ' + str(i) + ":", s)
    
print ('Raw attributions: ', attr)
value_increase = np.divide(s, s_init)
print ('% Increase in stake: ', value_increase/100)
norm_value_increase = value_increase / np.linalg.norm(value_increase, 1)
print ('Normalized value increase:', norm_value_increase)



1.6000000000000003
[[0.2  0.2  0.2  0.2  0.2 ]
 [0.2  0.16 0.2  0.2  0.2 ]
 [0.2  0.24 0.2  0.2  0.2 ]
 [0.2  0.2  0.2  0.16 0.2 ]
 [0.2  0.2  0.2  0.24 0.2 ]]
Initial stake: [ 1. 10.  1.  1.  1.]
Stake at 0: [ 1.01025352 10.09859155  1.01064789  1.00985915  1.01064789]
Stake at 1: [ 1.02061209 10.19815434  1.02140907  1.01981543  1.02140907]
Stake at 2: [ 1.0310768  10.29869793  1.03228474  1.02986979  1.03228474]
Stake at 3: [ 1.04164871 10.40023197  1.04327613  1.0400232   1.04327613]
Stake at 4: [ 1.05232894 10.50276621  1.05438447  1.05027662  1.05438447]
Stake at 5: [ 1.06311859 10.60631049  1.06561099  1.06063105  1.06561099]
Stake at 6: [ 1.07401878 10.71087477  1.07695695  1.07108748  1.07695695]
Stake at 7: [ 1.08503064 10.81646907  1.08842363  1.08164691  1.08842363]
Stake at 8: [ 1.09615532 10.92310353  1.10001231  1.09231035  1.10001231]
Stake at 9: [ 1.10739396 11.0307884   1.11172428  1.10307884  1.11172428]
Stake at 10: [ 1.11874775 11.139534    1.12356085  1.1139534   

In [123]:
import copy 

def normalize(M):
    result = {}
    sum_m = sum(M.values())
    for id in M.keys():
        if sum_m == 0:
             result[id] = 1 / len(M)
        else:
             result[id] = (M[id] / sum_m)
    return result

def multiply(A, B):
    result = {}
    for id in A.keys():
        result[id] = A[id] * B[id]
    return result       


class Bittensor:
    
    def __init__(self):
        self.S = {} # id --> stake
        self.W = {} # id --> weights (id, w)
        self.A = {} # id --> attr
        self.n = len(self.S.values())
        self.supply = sum(self.S.values())
        
    def init(self, id):
        self.S[id] = 0
        self.W[id] = None

    def set_weights(self, id, ids, weights):
        assert (len(weights) != 0)
        assert (len(ids) == len(weights))
        assert (abs(sum(weights) - 1.0) < 0.001)        
        self.W[id] = list(zip(ids, weights))
        
        
    def matmul(W, b):
        result = {}

        dem = (1 / len(W))

        for id in W.keys():
            result[id] = sum(b.values()) * dem

        for id in W.keys():
            if not W[id]:
                continue
            for w in W[id]:
                if w[0] == id:
                    result[id] -= (b[id] * (1 - w[1])) * dem
                else:            
                    result[w[0]] += (b[id] * w[1]) * dem

        return result

        
    def emit(self, steps=100):
        
        v = normalize(self.S)
        for _ in range(steps):
            v = matmul(self.W, v) 
        v = multiply(v, self.S)
        v = normalize(v)
        
        emmission = self.supply * 0.01 + 0.01
        for id in v:
            self.S[id] += emmission * v[id]

    def __str__(self):
        strng = "" 
        strng += "Stake: " + str([id + ": %.3f" % self.S[id] for id in self.S.keys()]) + "\n"
        strng += "Weights{ \n"
        for id in self.S.keys():
            strng += "\t"
            strng += str(id)
            strng += ": "
            if self.W[id] == None:
                strng += "nil"
                strng += " "
            else:
                for w in self.W[id]:
                    wstr = str(w[0]) + " " + str(w[1])
                    strng += wstr
                    strng += " "

            strng += "\n"
        strng += '}'
        return strng




SyntaxError: invalid syntax (<ipython-input-123-1fbcff800814>, line 1)

In [124]:
b = Bittensor()
b.init('a')
b.init('b')
b.init('c')
b.init('d')
b.init('e')


b.set_weights('a', ['a', 'b'], [0.9, 0.1])
b.set_weights('d', ['d', 'c'], [0.80, 0.2])
b.set_weights('c', ['c', 'b', 'a'], [0.5, 0.25, 0.25])
b.set_weights('e', ['e', 'd', 'c'], [0.6, 0.3, 0.1])

print (b)

for _ in range(1000):
    b.emit()
print (b)


Stake: ['a: 0.000', 'b: 0.000', 'c: 0.000', 'd: 0.000', 'e: 0.000']
Weights{ 
	a: a 0.9 b 0.1 
	b: nil 
	c: c 0.5 b 0.25 a 0.25 
	d: d 0.8 c 0.2 
	e: e 0.6 d 0.3 c 0.1 
}
Stake: ['a: 2.272', 'b: 2.957', 'c: 1.497', 'd: 2.095', 'e: 1.179']
Weights{ 
	a: a 0.9 b 0.1 
	b: nil 
	c: c 0.5 b 0.25 a 0.25 
	d: d 0.8 c 0.2 
	e: e 0.6 d 0.3 c 0.1 
}
